In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
# import pandas_profiling
import folium
from folium import plugins

In [2]:
# aisles = pd.read_csv("aisles.csv")
# departments = pd.read_csv("departments.csv")
order_products = pd.read_csv("order_products.csv")
orders = pd.read_csv("orders.csv")
# products = pd.read_csv("products.csv")

In [3]:
orders

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,NaN
1,2398795,1,2,3,7,15.0
2,473747,1,3,3,12,21.0
3,2254736,1,4,4,7,29.0
4,431534,1,5,4,15,28.0
...,...,...,...,...,...,...
3421078,2266710,206209,10,5,18,29.0
3421079,1854736,206209,11,4,10,30.0
3421080,626363,206209,12,1,12,18.0
3421081,2977660,206209,13,1,12,7.0


In [4]:
order_products_sample=order_products.sample(frac=0.1, random_state=None,axis=0)
order_products_sample.reset_index(drop=True, inplace=True)
order_products_sample

,order_id,product_id,add_to_cart_order
0,474796,26940,12
1,3025457,35140,3
2,1634599,44142,5
3,3239737,36023,15
4,2225035,28842,5
...,...,...,...
3381906,3283415,21938,10
3381907,1304612,7815,5
3381908,2960322,24489,25
3381909,3352927,25340,14


In [5]:
user_products = pd.merge(order_products_sample, orders[['order_id','user_id']], how='left', on='order_id')
user_products

,order_id,product_id,add_to_cart_order,user_id
0,474796,26940,12,62684
1,3025457,35140,3,59569
2,1634599,44142,5,20678
3,3239737,36023,15,115854
4,2225035,28842,5,151170
...,...,...,...,...
3381906,3283415,21938,10,30732
3381907,1304612,7815,5,40703
3381908,2960322,24489,25,159640
3381909,3352927,25340,14,15176


In [6]:
user_products_1=user_products[["user_id","product_id"]]
user_products_1

,user_id,product_id
0,62684,26940
1,59569,35140
2,20678,44142
3,115854,36023
4,151170,28842
...,...,...
3381906,30732,21938
3381907,40703,7815
3381908,159640,24489
3381909,15176,25340


In [7]:
user_products_1["min"]=user_products_1.index

D:\Anaconda3\envs\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [8]:
user_products_1

,user_id,product_id,min
0,62684,26940,0
1,59569,35140,1
2,20678,44142,2
3,115854,36023,3
4,151170,28842,4
...,...,...,...
3381906,30732,21938,3381906
3381907,40703,7815,3381907
3381908,159640,24489,3381908
3381909,15176,25340,3381909


In [9]:
grouped = user_products_1.groupby(["user_id","product_id"])["min"].aggregate( {'min','min'})
grouped_df=pd.DataFrame(grouped)
grouped_df

min
user_id product_id         
1       10258         34019
        12427        807060
        13032        170249
2       1559        1368398
        3151        1704171
...                     ...
206209  22713        912645
        23039       2616976
        28156       1023463
        40310       1618712
        40396        194333

[2682184 rows x 1 columns]

In [10]:
grouped_df["reordered"]=0

In [11]:
grouped_df

min  reordered
user_id product_id                    
1       10258         34019          0
        12427        807060          0
        13032        170249          0
2       1559        1368398          0
        3151        1704171          0
...                     ...        ...
206209  22713        912645          0
        23039       2616976          0
        28156       1023463          0
        40310       1618712          0
        40396        194333          0

[2682184 rows x 2 columns]

In [12]:
user_products_2 = pd.merge(user_products_1, grouped_df, how='left', on='min')
user_products_2

,user_id,product_id,min,reordered
0,62684,26940,0,0.0
1,59569,35140,1,0.0
2,20678,44142,2,0.0
3,115854,36023,3,0.0
4,151170,28842,4,0.0
...,...,...,...,...
3381906,30732,21938,3381906,0.0
3381907,40703,7815,3381907,NaN
3381908,159640,24489,3381908,0.0
3381909,15176,25340,3381909,0.0


In [13]:
user_products_2['reordered'].fillna(1, inplace=True)

In [14]:
user_products_2

,user_id,product_id,min,reordered
0,62684,26940,0,0.0
1,59569,35140,1,0.0
2,20678,44142,2,0.0
3,115854,36023,3,0.0
4,151170,28842,4,0.0
...,...,...,...,...
3381906,30732,21938,3381906,0.0
3381907,40703,7815,3381907,1.0
3381908,159640,24489,3381908,0.0
3381909,15176,25340,3381909,0.0
